In [ ]:
!pip install obonet
import obonet
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive


In [ ]:
# Mount Google Drive to access datasets
drive.mount('/content/drive')

Mounted at /content/drive


# Extracting Parent-Child Relationships from Gene Ontology Terms

In [ ]:
import json

def parse_go_obo(filename):
    with open(filename, 'r') as file:
        go_terms = {}
        term_details = {}
        for line in file:
            line = line.strip()
            if line == "[Term]" or line == "":
                if term_details:
                    # Store only the term ID and its parents
                    go_terms[term_details['id']] = term_details.get('parents', [])
                term_details = {}
            elif line.startswith("id:"):
                term_details['id'] = line.split("id: ")[1].strip()
            elif line.startswith("is_a:"):
                parent_id = line.split("is_a: ")[1].split("!")[0].strip()
                if 'parents' not in term_details:
                    term_details['parents'] = []
                term_details['parents'].append(parent_id)
        if term_details:
            # Ensure the last term is also captured
            go_terms[term_details['id']] = term_details.get('parents', [])

    return go_terms

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Parse the GO terms and extract only the parents
go_terms = parse_go_obo('/content/drive/My Drive/DATASET/Train/go-basic.obo')

# Save the parsed terms to a JSON file
save_to_json(go_terms, '/content/drive/My Drive/DATASET/Train/go_terms_parents.json')

# Optionally print each term and its parents (for quick check)
for term_id, parents in go_terms.items():
    print(f"GO Term: {term_id}, Parents: {parents}")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
GO Term: GO:1903059, Parents: ['GO:0031399', 'GO:0050746', 'GO:2000112']
GO Term: GO:1903060, Parents: ['GO:0031400', 'GO:0050748', 'GO:1903059', 'GO:2000113']
GO Term: GO:1903061, Parents: ['GO:0031401', 'GO:0050747', 'GO:1903059']
GO Term: GO:1903062, Parents: ['GO:0032374']
GO Term: GO:1903063, Parents: ['GO:0032375', 'GO:1903062']
GO Term: GO:1903064, Parents: ['GO:0032376', 'GO:1903062']
GO Term: GO:1903065, Parents: []
GO Term: GO:1903066, Parents: ['GO:0032880']
GO Term: GO:1903067, Parents: ['GO:1903066', 'GO:1903828']
GO Term: GO:1903068, Parents: ['GO:1903066', 'GO:1903829']
GO Term: GO:1903069, Parents: ['GO:0032434', 'GO:1904292']
GO Term: GO:1903070, Parents: ['GO:0032435', 'GO:1903069', 'GO:1904293']
GO Term: GO:1903071, Parents: ['GO:0032436', 'GO:1903069', 'GO:1904294']
GO Term: GO:1903072, Parents: ['GO:0043254']
GO Term: GO:1903073, Parents: ['GO:0031333', 'GO:1903072', 'GO:2001237']
GO Term

In [ ]:
# Define the path to the pickle file
pickle_path = "/content/drive/My Drive/DATASET/labels_df.pkl"

# load labels_df from the pickle file
labels_df = pd.read_pickle(pickle_path)
print("Loaded labels_df from pickle.")

# Display the DataFrame to visually inspect its content
display(labels_df.head())


Loaded labels_df from pickle.


,GO:0005575,GO:0008150,GO:0110165,GO:0003674,GO:0005622,GO:0009987,GO:0043226,GO:0043229,GO:0005488,GO:0043227,...,GO:0034250,GO:0140053,GO:0031345,GO:0098802,GO:0045861,GO:0051783,GO:0031674,GO:0001818,GO:0006874,GO:0016887
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import json
allowed_labels = set(labels_df.columns)  # Extraction des GO terms du DataFrame

# Charger les données JSON des termes GO
with open('/content/drive/My Drive/DATASET/go_terms_parents.json', 'r') as file:
    go_terms_parents = json.load(file)

# Filtrer les termes GO pour ne conserver que ceux présents dans allowed_labels
filtered_go_terms = {term: [parent for parent in parents if parent in allowed_labels]
                     for term, parents in go_terms_parents.items() if term in allowed_labels or any(parent in allowed_labels for parent in parents)}

# Sauvegarder les termes GO filtrés dans un nouveau fichier JSON
with open('/content/drive/My Drive/DATASET/Train/filtered0_go_terms_parents.json', 'w') as file:
    json.dump(filtered_go_terms, file, indent=4)

In [ ]:
import numpy as np
import pandas as pd
import json
import networkx as nx

# Chargement des données JSON des termes GO et leurs parents
with open('/content/drive/My Drive/DATASET/Train/filtered0_go_terms_parents.json', 'r') as file:
    filtered_go_terms = json.load(file)

# Chargement et conversion du DataFrame des labels en booléen
labels_df = pd.read_pickle("/content/drive/My Drive/DATASET/labels_df.pkl").astype(bool)

# Création d'un graphe dirigé
G = nx.DiGraph()

# Ajout des relations dans le graphe
for child, parents in filtered_go_terms.items():
    for parent in parents:
        G.add_edge(parent, child)  # Ajoute un arc du parent vers l'enfant

# Application de la propagation
for node in G:
    if G.in_degree(node) > 0 and node in labels_df.columns:  # Si le noeud a des parents et est dans labels_df
        # Cumuler les étiquettes des parents
        parent_labels = np.any([labels_df.loc[:, parent].values for parent in G.predecessors(node) if parent in labels_df.columns], axis=0)
        # Mettre à jour l'étiquette du noeud
        labels_df.loc[:, node] = labels_df.loc[:, node] | parent_labels

# Sauvegarde du DataFrame mis à jour
updated00_labels_df_path = "/content/drive/My Drive/DATASET/updated00_labels_df.pkl"
labels_df.to_pickle(updated00_labels_df_path)
print(f"Updated labels DataFrame saved to {updated00_labels_df_path}")


Updated labels DataFrame saved to /content/drive/My Drive/DATASET/updated00_labels_df.pkl
